In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from scipy.stats import pearsonr
import itertools
from statsmodels.tsa.stattools import kpss
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.stattools import adfuller
import dask.dataframe as dd

In [3]:
pred = pd.read_csv('predictions.csv')

In [5]:
pred.head()

,store_nbr,product_category,onpromotion,year,month,day,y
0,0,0,-0.505838,0.0,-0.707107,0.330771,-0.427458
1,0,0,-0.505838,0.0,-0.707107,1.025389,-0.351236
2,0,0,-0.505838,0.0,1.414214,-1.356159,-0.406578
3,0,1,-0.505838,0.0,-0.707107,0.330771,-0.407033
4,0,1,-0.505838,0.0,-0.707107,1.025389,0.595406


In [8]:
y_mean = 4483330.01
y_std = 1401420.23

# Apply the inverse transformation formula
pred['y'] = (pred['y'] * y_std) + y_mean

In [9]:
pred.head()

,store_nbr,product_category,onpromotion,year,month,day,y
0,0,0,-0.505838,0.0,-0.707107,0.330771,3.884282e+06
1,0,0,-0.505838,0.0,-0.707107,1.025389,3.991100e+06
2,0,0,-0.505838,0.0,1.414214,-1.356159,3.913544e+06
3,0,1,-0.505838,0.0,-0.707107,0.330771,3.912906e+06
4,0,1,-0.505838,0.0,-0.707107,1.025389,5.317745e+06


In [11]:
test_df = pd.read_csv('test.csv')

In [16]:
test_df = test_df.rename(columns={'family': 'product_category'})


In [17]:
test_df.head()

,date,store_nbr,product_category,onpromotion
0,2017-08-16,1,AUTOMOTIVE,0
1,2017-08-16,1,BABY CARE,0
2,2017-08-16,1,BEAUTY,2
3,2017-08-16,1,BEVERAGES,20
4,2017-08-16,1,BOOKS,0


In [18]:
test_df['date'] = pd.to_datetime(test_df['date'])
test_df = test_df.set_index('date')

test_df = test_df.rename(columns={'family': 'product_category'})
test_df = test_df.groupby(['store_nbr', 'product_category'])['onpromotion'].resample('W').sum().reset_index()

print(test_df.head())

   store_nbr product_category       date  onpromotion
0          1       AUTOMOTIVE 2017-08-20            0
1          1       AUTOMOTIVE 2017-08-27            0
2          1       AUTOMOTIVE 2017-09-03            0
3          1        BABY CARE 2017-08-20            0
4          1        BABY CARE 2017-08-27            0


In [20]:
test_df['y'] = pred['y']


In [22]:
test_df.head()

,store_nbr,product_category,date,onpromotion,y
0,1,AUTOMOTIVE,2017-08-20,0,3.884282e+06
1,1,AUTOMOTIVE,2017-08-27,0,3.991100e+06
2,1,AUTOMOTIVE,2017-09-03,0,3.913544e+06
3,1,BABY CARE,2017-08-20,0,3.912906e+06
4,1,BABY CARE,2017-08-27,0,5.317745e+06


In [29]:
test_df['y'] = (test_df['y'] / 1e6).round(2)


In [31]:
test_df = test_df.rename(columns={'y': 'sales_predicted'})


In [33]:
test_df = test_df.rename(columns={'sales_predicted': 'sales_predicted_in_millions'})


In [34]:
test_df.head()

,store_nbr,product_category,date,onpromotion,sales_predicted_in_millions
0,1,AUTOMOTIVE,2017-08-20,0,3.88
1,1,AUTOMOTIVE,2017-08-27,0,3.99
2,1,AUTOMOTIVE,2017-09-03,0,3.91
3,1,BABY CARE,2017-08-20,0,3.91
4,1,BABY CARE,2017-08-27,0,5.32


In [35]:
test_df.to_csv('predictions.csv', index=False)
